In [148]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import yfinance as yf
import hvplot.pandas
from pathlib import Path
import requests
import json
import csv
import io
from finta import TA

In [149]:
trading_days_year = 252
total_days_year = 365
thirty_years_ago = (pd.Timestamp.today() - pd.Timedelta(days = total_days_year * 30)).date()
ten_years_ago = (pd.Timestamp.today() - pd.Timedelta(days = total_days_year * 10)).date()
five_years_ago = (pd.Timestamp.today() - pd.Timedelta(days = total_days_year * 5)).date()
three_years_ago = (pd.Timestamp.today() - pd.Timedelta(days = total_days_year * 3)).date()
one_year = (pd.Timestamp.today() - pd.Timedelta(days = total_days_year * 1)).date()
six_months = (pd.Timestamp.today() - pd.Timedelta(days = total_days_year / 2)).date()
yesterday = (pd.Timestamp.today() - pd.Timedelta(days = 1)).date()

In [150]:
def get_close(start, end, tickers = 'spy', interval = '1d'):
    
    if len(tickers) < 5:
        df = yf.Ticker(tickers)
        data = df.history(start=start, end=end, interval = interval)
        data = data.loc[:,['Close']]
        #data = data.rename(tickers)
    else:
        df = yf.Tickers(tickers)
        data = df.history(start=start, end=end, interval = interval)
        data = data.loc[:,['Close']]
        
    data.index = data.index.date
    return data
def get_data(start, end, tickers = 'spy', interval = '1d'):
    
    if len(tickers) < 5:
        df = yf.Ticker(tickers)
        data = df.history(start=start, end=end, interval = interval)
    else:
        df = yf.Tickers(tickers)
        data = df.history(start=start, end=end, interval = interval)
        
    data.index = data.index.date
    return data

# This algo will go through every single crossover strategy, using SPY data for the last 30 years

### Algo will buy when shorter / faster EMA will cross over slower EMA and sell when it crosses below

In [151]:
df = get_data(thirty_years_ago, yesterday)
df.drop(columns= ['Dividends','Stock Splits','Capital Gains'], inplace= True)
columns = ['Strategy', 'Annualized Return', 'Cumulative Return', 'Annual Volatility', 'Sharpe Ratio']
portfolio_eval_df = pd.DataFrame( columns= columns)

initial_capital = 100000.0
share_size = 500
sqrt = np.sqrt(252)

for short_ma in range(2, 200):
    for long_ma in range(3, 201):
        if long_ma > short_ma:
            short_window = short_ma
            long_window = long_ma
            df['Short'] = TA.EMA(df, short_window)
            df['Long'] = TA.EMA(df, long_window)

            # signal columns to calculate trade returns

            df['Signal'] = 0.0
            df['Signal'][short_window:] = np.where(df['Short'][short_window:] > df['Long'][short_window:], 1.0, 0.0)
            df['Entry/Exit'] = df['Signal'].diff()
            df["Position"] = share_size * df['Signal']
            df["Entry/Exit Position"] = df["Position"].diff()
            df["Portfolio Holdings"] = df["Position"] * df['Close']
            df["Portfolio Cash"] = initial_capital - (df['Close'] * df['Entry/Exit Position']).cumsum()
            df["Portfolio Total"] = df["Portfolio Cash"] + df["Portfolio Holdings"]
            df["Portfolio Daily Returns"] = df["Portfolio Total"].pct_change()
            df["Portfolio Cumulative Returns"] = (1 + df["Portfolio Daily Returns"]).cumprod()

            # append evaluation metrics to a df as a new row
            ann_ret =  df['Portfolio Daily Returns'].mean() * 252
            cum_ret = df['Portfolio Cumulative Returns'][-1]
            ann_volat = df['Portfolio Daily Returns'].std() * sqrt
            sharpe = ann_ret / ann_volat

            new_row = {'Strategy' : f'{short_window} & {long_window}',
                       'Annualized Return' : ann_ret,
                       'Cumulative Return' : cum_ret,
                       'Annual Volatility' : ann_volat,
                       'Sharpe Ratio' : sharpe}
            portfolio_eval_df = portfolio_eval_df.append(new_row, ignore_index=True)
            
            print(f'{short_window} & {long_window} done')

2 & 3 done
2 & 4 done
2 & 5 done
2 & 6 done
2 & 7 done
2 & 8 done
2 & 9 done
2 & 10 done
2 & 11 done
2 & 12 done
2 & 13 done
2 & 14 done
2 & 15 done
2 & 16 done
2 & 17 done
2 & 18 done
2 & 19 done
2 & 20 done
2 & 21 done
2 & 22 done
2 & 23 done
2 & 24 done
2 & 25 done
2 & 26 done
2 & 27 done
2 & 28 done
2 & 29 done
2 & 30 done
2 & 31 done
2 & 32 done
2 & 33 done
2 & 34 done
2 & 35 done
2 & 36 done
2 & 37 done
2 & 38 done
2 & 39 done
2 & 40 done
2 & 41 done
2 & 42 done
2 & 43 done
2 & 44 done
2 & 45 done
2 & 46 done
2 & 47 done
2 & 48 done
2 & 49 done
2 & 50 done
2 & 51 done
2 & 52 done
2 & 53 done
2 & 54 done
2 & 55 done
2 & 56 done
2 & 57 done
2 & 58 done
2 & 59 done
2 & 60 done
2 & 61 done
2 & 62 done
2 & 63 done
2 & 64 done
2 & 65 done
2 & 66 done
2 & 67 done
2 & 68 done
2 & 69 done
2 & 70 done
2 & 71 done
2 & 72 done
2 & 73 done
2 & 74 done
2 & 75 done
2 & 76 done
2 & 77 done
2 & 78 done
2 & 79 done
2 & 80 done
2 & 81 done
2 & 82 done
2 & 83 done
2 & 84 done
2 & 85 done
2 & 86 done

## Results sorted by Cumulative Return

In [159]:

portfolio_eval_df.sort_values('Cumulative Return', ascending= False).head(30)


,Strategy,Annualized Return,Cumulative Return,Annual Volatility,Sharpe Ratio
19700,199 & 200,0.038659,2.985592,0.065249,0.592494
19421,176 & 197,0.038629,2.984129,0.065026,0.594051
19374,174 & 199,0.038627,2.983946,0.065032,0.593968
19443,177 & 196,0.038625,2.983763,0.065038,0.593887
19398,175 & 198,0.038625,2.983763,0.065038,0.593886
19349,173 & 200,0.038622,2.983397,0.065051,0.593722
19521,181 & 192,0.038603,2.981256,0.065124,0.592756
19484,179 & 194,0.038599,2.980798,0.065140,0.592548
19538,182 & 191,0.038596,2.980524,0.065149,0.592430
19569,184 & 189,0.038596,2.980432,0.065152,0.592389


## Results sorted by Sharpe Ratio

In [160]:
portfolio_eval_df.sort_values('Sharpe Ratio', ascending= False).head(30)

,Strategy,Annualized Return,Cumulative Return,Annual Volatility,Sharpe Ratio
888,6 & 109,0.034828,2.712097,0.054918,0.634182
4322,25 & 47,0.034761,2.705775,0.055131,0.630526
1070,7 & 98,0.034468,2.683871,0.054722,0.629873
1069,7 & 97,0.034482,2.684835,0.054759,0.629705
6187,36 & 53,0.035268,2.741786,0.056293,0.626510
4496,26 & 47,0.034671,2.697303,0.055391,0.625930
4149,24 & 49,0.034563,2.689160,0.055260,0.625458
3976,23 & 52,0.034545,2.687295,0.055348,0.624131
889,6 & 110,0.034578,2.689626,0.055425,0.623866
4323,25 & 48,0.034512,2.684448,0.055404,0.622923


## Comparing it with buy and hold

In [162]:
spy = get_close(thirty_years_ago, yesterday)

buy_and_hold_return = (1+ spy.pct_change()).cumprod()
buy_and_hold_return

,Close
1993-06-23,NaN
1993-06-24,1.013428
1993-06-25,1.012721
1993-06-28,1.024028
1993-06-29,1.019081
...,...
2023-06-08,16.774294
2023-06-09,16.804392
2023-06-12,16.956839
2023-06-13,17.068634


# Conclusion
## None of the crossover strategies outperform buy and hold, which returned over 1,700 %, compared to the most succesful crossover strategy returning 298 %, which is only 3.8% annualized, much less than the risk-free rate